<a href="https://colab.research.google.com/github/KashinathJ/Assignment-/blob/main/Assessment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
# Install necessary packages (including flask_ngrok, though we won't use it directly now)
!pip install flask pyngrok pandas nest_asyncio flask_ngrok

# --- Ngrok Configuration ---
# 🚨 IMPORTANT: Replace 'YOUR_REAL_NGROK_AUTH_TOKEN_HERE' with your actual ngrok token.
import os
from pyngrok import ngrok, conf
import nest_asyncio

# 1. Set the token
# --- Make sure this is your actual ngrok token! ---
NGROK_AUTH_TOKEN = "YOUR_REAL_NGROK_AUTH_TOKEN_HERE"
conf.get_default().auth_token = NGROK_AUTH_TOKEN

# 2. Apply nest_asyncio for Colab compatibility
nest_asyncio.apply()

print(f"✅ Libraries installed. Ngrok token set (Ensure the token is correct!).")

✅ Libraries installed. Ngrok token set (Ensure the token is correct!).


In [15]:
from flask import Flask, request, jsonify
from datetime import datetime, timedelta
import uuid
import json

app = Flask(__name__)

# --- 1. In-Memory Data Stores ---
rooms = {
    "101": {"id": "101", "name": "The Executive Boardroom", "baseHourlyRate": 500, "capacity": 10},
    "102": {"id": "102", "name": "The Focus Pod", "baseHourlyRate": 300, "capacity": 4},
    "103": {"id": "103", "name": "The Training Suite", "baseHourlyRate": 750, "capacity": 20},
    "104": {"id": "104", "name": "The Quiet Corner", "baseHourlyRate": 400, "capacity": 6},
}
# Status: CONFIRMED, CANCELLED
bookings = {}

# --- Helper Functions for Business Logic ---

def is_peak_hour(dt: datetime) -> bool:
    """Checks if a given datetime falls within peak hours."""
    day_of_week = dt.weekday() # Monday=0, Sunday=6
    hour = dt.hour
    is_weekday = 0 <= day_of_week <= 4
    peak_morning = 10 <= hour < 13
    peak_afternoon = 16 <= hour < 19
    return is_weekday and (peak_morning or peak_afternoon)

def calculate_price(room_id: str, start_time: datetime, end_time: datetime) -> float:
    """Calculates the dynamic price for a booking based on 30-min slots."""
    room = rooms[room_id]
    base_rate = room['baseHourlyRate']
    total_price = 0.0

    current_time = start_time
    while current_time < end_time:
        next_time = current_time + timedelta(minutes=30)
        if next_time > end_time:
            next_time = end_time

        duration_hours = (next_time - current_time).total_seconds() / 3600
        rate_multiplier = 1.5 if is_peak_hour(current_time) else 1.0

        slot_price = base_rate * rate_multiplier * duration_hours
        total_price += slot_price

        current_time = next_time

    return round(total_price, 2)

def check_conflict(room_id: str, new_start: datetime, new_end: datetime) -> str:
    """Checks for time conflicts with existing CONFIRMED bookings."""
    for booking in bookings.values():
        if booking['roomId'] == room_id and booking['status'] == 'CONFIRMED':
            existing_start = booking['startTime']
            existing_end = booking['endTime']
            if (new_start < existing_end) and (new_end > existing_start):
                return f"Room already booked from {existing_start.strftime('%I:%M %p')} to {existing_end.strftime('%I:%M %p')} UTC"
    return None

# --- API Endpoints ---

@app.route('/api/rooms', methods=['GET'])
def list_rooms():
    return jsonify(list(rooms.values()))

@app.route('/api/bookings', methods=['POST'])
def create_booking():
    data = request.json
    try:
        room_id = data['roomId']
        user_name = data['userName']
        start_time = datetime.fromisoformat(data['startTime'].replace('Z', '+00:00'))
        end_time = datetime.fromisoformat(data['endTime'].replace('Z', '+00:00'))
    except (KeyError, ValueError):
        return jsonify({"error": "Invalid input format."}), 400

    duration = end_time - start_time
    if start_time >= end_time or duration > timedelta(hours=12) or duration <= timedelta(0):
        return jsonify({"error": "Invalid booking duration. Must be positive and <= 12 hours."}), 400

    conflict_message = check_conflict(room_id, start_time, end_time)
    if conflict_message:
        return jsonify({"error": conflict_message}), 409

    total_price = calculate_price(room_id, start_time, end_time)
    booking_id = str(uuid.uuid4())
    new_booking = {
        "bookingId": booking_id,
        "roomId": room_id,
        "userName": user_name,
        "startTime": start_time,
        "endTime": end_time,
        "totalPrice": total_price,
        "status": "CONFIRMED",
        "createdAt": datetime.utcnow()
    }
    bookings[booking_id] = new_booking

    response_data = {
        k: v.isoformat().replace('+00:00', 'Z') if isinstance(v, datetime) else v
        for k, v in new_booking.items()
    }
    return jsonify(response_data), 201

@app.route('/api/bookings/<id>/cancel', methods=['POST'])
def cancel_booking_endpoint(id):
    booking = bookings.get(id)
    if not booking:
        return jsonify({"error": "Booking not found."}), 404

    if booking['status'] == 'CANCELLED':
        return jsonify({"error": "Booking is already cancelled."}), 400

    start_time = booking['startTime']
    time_remaining = start_time - datetime.utcnow()

    if time_remaining <= timedelta(hours=2):
        return jsonify({"error": "Cancellation policy violation: Must cancel more than 2 hours before the start time."}), 400

    booking['status'] = 'CANCELLED'
    bookings[id] = booking

    return jsonify({"bookingId": id, "status": "CANCELLED", "message": "Booking successfully cancelled."})

@app.route('/api/analytics', methods=['GET'])
def get_analytics():
    from_date_str = request.args.get('from')
    to_date_str = request.args.get('to')

    if not from_date_str or not to_date_str:
        return jsonify({"error": "Missing 'from' and 'to' date parameters."}), 400

    try:
        from_dt = datetime.strptime(from_date_str, '%Y-%m-%d')
        to_dt = datetime.strptime(to_date_str, '%Y-%m-%d') + timedelta(days=1)
    except ValueError:
        return jsonify({"error": "Invalid date format. Use YYYY-MM-DD."}), 400

    analytics_data = {}

    for booking in bookings.values():
        if booking['status'] == 'CONFIRMED':
            booking_start = booking['startTime']

            if from_dt <= booking_start < to_dt:
                room_id = booking['roomId']
                duration_hours = (booking['endTime'] - booking_start).total_seconds() / 3600
                total_revenue = booking['totalPrice']

                if room_id not in analytics_data:
                    analytics_data[room_id] = {
                        "roomId": room_id,
                        "roomName": rooms[room_id]['name'],
                        "totalHours": 0.0,
                        "totalRevenue": 0.0
                    }

                analytics_data[room_id]['totalHours'] += duration_hours
                analytics_data[room_id]['totalRevenue'] += total_revenue

    result = []
    for data in analytics_data.values():
         result.append({
            "roomId": data['roomId'],
            "roomName": data['roomName'],
            "totalHours": round(data['totalHours'], 2),
            "totalRevenue": round(data['totalRevenue'], 2)
        })

    return jsonify(result)

@app.route('/api/admin/bookings', methods=['GET'])
def list_all_bookings():
    """Admin endpoint to list all bookings (CONFIRMED and CANCELLED)."""
    response_list = []
    sorted_bookings = sorted(bookings.values(), key=lambda b: b['startTime'], reverse=True)

    for booking in sorted_bookings:
        response_data = {
            k: v.isoformat().replace('+00:00', 'Z') if isinstance(v, datetime) else v
            for k, v in booking.items()
        }
        room = rooms.get(booking['roomId'], {})
        response_data['roomName'] = room.get('name', 'N/A')
        response_list.append(response_data)

    return jsonify(response_list)

# The / route is defined below, after frontend_html is created.

In [16]:
# Declare frontend_html globally so the final cell can update the URL
global frontend_html
API_URL_PLACEHOLDER = "NGROK_URL_PLACEHOLDER"

# NOTE: All literal JS braces {{...}} are doubled in the Python f-string
frontend_html = f"""
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Workspace Booking System</title>
    <style>
        body {{ {{ font-family: sans-serif; margin: 20px; background-color: #f4f4f9; }} }}
        h1 {{ {{ color: #333; }} }}
        .container {{ {{ display: flex; gap: 20px; max-width: 1200px; margin: auto; }} }}
        .section {{ {{ background: white; padding: 20px; border-radius: 8px; box-shadow: 0 2px 4px rgba(0,0,0,0.1); flex-grow: 1; }} }}
        .room-card {{ {{ border: 1px solid #ddd; padding: 10px; margin-bottom: 10px; border-radius: 4px; background: #e9e9e9; }} }}
        .error {{ {{ color: red; font-weight: bold; }} }}
        .success {{ {{ color: green; font-weight: bold; }} }}
        table {{ {{ width: 100%; border-collapse: collapse; margin-top: 10px; }} }}
        th, td {{ {{ border: 1px solid #ddd; padding: 8px; text-align: left; }} }}
        th {{ {{ background-color: #333; color: white; }} }}
        input[type="datetime-local"] {{ {{ padding: 8px; border: 1px solid #ccc; border-radius: 4px; }} }}
    </style>
</head>
<body>
    <h1>Workspace Booking System</h1>
    <p>Backend URL: <strong id="backendUrlDisplay">{API_URL_PLACEHOLDER}</strong></p>

    <div class="container">
        <div class="section" style="flex-basis: 30%;">
            <h2>1. Available Rooms</h2>
            <div id="roomsList">Loading Rooms...</div>

            <hr>

            <h2>2. Book a Room</h2>
            <form id="bookingForm">
                <p>
                    <label>Room:</label>
                    <select id="roomId" name="roomId" required></select>
                </p>
                <p>
                    <label>Your Name:</label>
                    <input type="text" id="userName" required>
                </p>
                <p>
                    <label>Start Time (Local Input, Sent as UTC):</label>
                    <input type="datetime-local" id="startTime" step="1800" required>
                </p>
                <p>
                    <label>End Time (Local Input, Sent as UTC):</label>
                    <input type="datetime-local" id="endTime" step="1800" required>
                </p>
                <button type="submit">Create Booking</button>
            </form>
            <div id="bookingResult" style="margin-top: 10px;"></div>
        </div>

        <div class="section" style="flex-basis: 70%;">
            <h2>3. Admin View: Bookings & Analytics</h2>

            <div style="margin-bottom: 20px;">
                <h3>Analytics Query</h3>
                <form id="analyticsForm" style="display: flex; gap: 10px;">
                    <label>From: <input type="date" id="analyticsFrom" required></label>
                    <label>To: <input type="date" id="analyticsTo" required></label>
                    <button type="submit">Fetch Analytics</button>
                </form>
                <div id="analyticsOutput" style="margin-top: 10px;"></div>
            </div>

            <hr>

            <h3>All Bookings (Active & Cancelled)</h3>
            <div id="bookingsList">Loading Bookings...</div>
        </div>
    </div>

    <script>
        const API_BASE = document.getElementById('backendUrlDisplay').textContent + "/api";

        // --- Helper: Format Datetime to ISO String (Z format) ---
        function formatDateTimeToISO(dtLocalString) {{
            if (!dtLocalString) return null;
            const date = new Date(dtLocalString);
            return date.toISOString();
        }}

        // --- 1. Fetch Rooms and Populate UI ---
        async function fetchRooms() {{
            try {{
                const response = await fetch(`${{API_BASE}}/rooms`);
                const data = await response.json();

                const roomsListDiv = document.getElementById('roomsList');
                roomsListDiv.innerHTML = data.map(room => `
                    <div class="room-card">
                        <strong>${{room.name}}</strong> (Capacity: ${{room.capacity}})<br>
                        Base Rate: \${{room.baseHourlyRate}} / hr
                        <br><small>Peak Rate: \${{room.baseHourlyRate * 1.5}} / hr</small>
                    </div>
                `).join('');

                const roomSelect = document.getElementById('roomId');
                roomSelect.innerHTML = data.map(room => `
                    <option value="${{room.id}}">${{room.name}} (\${{room.baseHourlyRate}}/hr)</option>
                `).join('');

            }} catch (error) {{
                console.error("Error fetching rooms:", error);
                document.getElementById('roomsList').innerHTML = `<p class="error">Failed to load rooms.</p>`;
            }}
        }}

        // --- 2. Create Booking ---
        document.getElementById('bookingForm').addEventListener('submit', async function(e) {{
            e.preventDefault();
            const resultDiv = document.getElementById('bookingResult');
            resultDiv.innerHTML = 'Processing...';

            const bookingData = {{
                roomId: document.getElementById('roomId').value,
                userName: document.getElementById('userName').value,
                startTime: formatDateTimeToISO(document.getElementById('startTime').value),
                endTime: formatDateTimeToISO(document.getElementById('endTime').value)
            }};

            try {{
                const response = await fetch(`${{API_BASE}}/bookings`, {{
                    method: 'POST',
                    headers: {{ 'Content-Type': 'application/json' }},
                    body: JSON.stringify(bookingData)
                }});

                const data = await response.json();

                if (response.ok) {{
                    resultDiv.className = 'success';
                    resultDiv.innerHTML = `
                        <strong>Booking CONFIRMED!</strong><br>
                        ID: ${{data.bookingId.substring(0, 8)}}...<br>
                        Room: ${{data.roomId}}<br>
                        Total Price: \${{data.totalPrice}}<br>
                        Status: ${{data.status}}
                    `;
                    fetchBookings(); // Refresh admin list
                }} else {{
                    resultDiv.className = 'error';
                    resultDiv.innerHTML = `<strong>Booking Failed:</strong> ${{data.error || 'Unknown Error'}}`;
                }}
            }} catch (error) {{
                resultDiv.className = 'error';
                resultDiv.innerHTML = `<strong>Network Error:</strong> Cannot connect to API.`;
                console.error("Booking submission error:", error);
            }}
        }});

        // --- 3. Cancel Booking ---
        async function cancelBooking(bookingId) {{
            if (!confirm(`Are you sure you want to cancel booking ${{bookingId.substring(0, 8)}}...?`)) {{
                return;
            }}

            try {{
                const response = await fetch(`${{API_BASE}}/bookings/${{bookingId}}/cancel`, {{
                    method: 'POST'
                }});

                const data = await response.json();

                if (response.ok) {{
                    alert(`Cancellation successful: ${{data.message}}`);
                    fetchBookings(); // Refresh list
                }} else {{
                    alert(`Cancellation failed: ${{data.error || 'Unknown error'}}`);
                }}
            }} catch (error) {{
                alert('Network Error during cancellation.');
                console.error("Cancellation error:", error);
            }}
        }}

        // --- 4. Fetch All Bookings (for Admin View) ---
        async function fetchBookings() {{
            const response = await fetch(`${{API_BASE}}/admin/bookings`);
            if (!response.ok) {{
                document.getElementById('bookingsList').innerHTML = `<p class="error">Failed to load bookings.</p>`;
                return;
            }}
            const data = await response.json();

            const listDiv = document.getElementById('bookingsList');
            if (data.length === 0) {{
                listDiv.innerHTML = `<p>No bookings found.</p>`;
                return;
            }}

            listDiv.innerHTML = `
                <table>
                    <thead>
                        <tr>
                            <th>ID</th><th>Room</th><th>User</th><th>Start (UTC)</th><th>End (UTC)</th><th>Price</th><th>Status</th><th>Actions</th>
                        </tr>
                    </thead>
                    <tbody>
                        ${{data.map(b => `
                            <tr>
                                <td>${{b.bookingId.substring(0, 8)}}...</td>
                                <td>${{b.roomName}}</td>
                                <td>${{b.userName}}</td>
                                <td>${{new Date(b.startTime).toLocaleString('en-US', {{timeZone: 'UTC', timeStyle: 'short', dateStyle: 'short'}})}}</td>
                                <td>${{new Date(b.endTime).toLocaleString('en-US', {{timeZone: 'UTC', timeStyle: 'short', dateStyle: 'short'}})}}</td>
                                <td>\${{b.totalPrice}}</td>
                                <td style="color: ${{b.status === 'CONFIRMED' ? 'green' : 'red'}};">${{b.status}}</td>
                                <td>
                                    ${{b.status === 'CONFIRMED' ?
                                        `<button onclick="cancelBooking('${{b.bookingId}}')">Cancel</button>` :
                                        'N/A'
                                    }}
                                </td>
                            </tr>
                        `).join('')}}
                    </tbody>
                </table>
            `;
        }}

        // --- 4. Analytics Report ---
        document.getElementById('analyticsForm').addEventListener('submit', async function(e) {{
            e.preventDefault();
            const from = document.getElementById('analyticsFrom').value;
            const to = document.getElementById('analyticsTo').value;
            const outputDiv = document.getElementById('analyticsOutput');
            outputDiv.innerHTML = 'Generating report...';

            try {{
                const response = await fetch(`${{API_BASE}}/analytics?from=${{from}}&to=${{to}}`);
                const data = await response.json();

                if (response.ok) {{
                    if (data.length === 0) {{
                        outputDiv.innerHTML = `<p class="error">No CONFIRMED revenue data found for this period.</p>`;
                        return;
                    }}

                    const totalRevenue = data.reduce((sum, item) => sum + item.totalRevenue, 0);

                    outputDiv.innerHTML = `
                        <h4>Revenue Report (${{from}} to ${{to}})</h4>
                        <strong>Total Revenue: \${{totalRevenue.toFixed(2)}}</strong>
                        <table>
                            <thead>
                                <tr><th>Room</th><th>Total Hours</th><th>Total Revenue</th></tr>
                            </thead>
                            <tbody>
                                ${{data.map(item => `
                                    <tr>
                                        <td>${{item.roomName}}</td>
                                        <td>${{item.totalHours}} hrs</td>
                                        <td>\${{item.totalRevenue}}</td>
                                    </tr>
                                `).join('')}}
                            </tbody>
                        </table>
                    `;
                }} else {{
                    outputDiv.className = 'error';
                    outputDiv.innerHTML = `<strong>Analytics Failed:</strong> ${{data.error || 'Unknown Error'}}`;
                }}

            }} catch (error) {{
                outputDiv.className = 'error';
                outputDiv.innerHTML = `<strong>Network Error:</strong> Failed to fetch analytics.`;
                console.error("Analytics error:", error);
            }}
        }});

        // --- Initialize on Load ---
        document.addEventListener('DOMContentLoaded', () => {{
            fetchRooms();
            fetchBookings();
            document.getElementById('analyticsFrom').value = '2020-01-01';
            document.getElementById('analyticsTo').value = '2030-12-31';
        }});
    </script>
</body>
</html>
"""

<>:290: SyntaxWarning: invalid escape sequence '\$'
<>:290: SyntaxWarning: invalid escape sequence '\$'
<>:289: SyntaxWarning: invalid escape sequence '\$'
<>:290: SyntaxWarning: invalid escape sequence '\$'
<>:290: SyntaxWarning: invalid escape sequence '\$'
<>:290: SyntaxWarning: invalid escape sequence '\$'
<>:290: SyntaxWarning: invalid escape sequence '\$'
<>:290: SyntaxWarning: invalid escape sequence '\$'
<>:290: SyntaxWarning: invalid escape sequence '\$'
<>:289: SyntaxWarning: invalid escape sequence '\$'
<>:290: SyntaxWarning: invalid escape sequence '\$'
<>:290: SyntaxWarning: invalid escape sequence '\$'
<>:290: SyntaxWarning: invalid escape sequence '\$'
<>:290: SyntaxWarning: invalid escape sequence '\$'
/tmp/ipython-input-1142565755.py:290: SyntaxWarning: invalid escape sequence '\$'
/tmp/ipython-input-1142565755.py:290: SyntaxWarning: invalid escape sequence '\$'
/tmp/ipython-input-1142565755.py:289: SyntaxWarning: invalid escape sequence '\$'
  """
/tmp/ipython-input-1

In [17]:
# Define the root route here, after frontend_html is available
@app.route('/')
def home():
    """Serves the main HTML/JS Frontend."""
    global frontend_html
    return frontend_html

# Start ngrok tunnel and Flask server
try:
    # 1. Start the ngrok tunnel on port 5000
    public_url = ngrok.connect(5000).public_url

    # 2. Update the frontend HTML with the actual ngrok URL
    global frontend_html
    frontend_html = frontend_html.replace(API_URL_PLACEHOLDER, public_url)

    # 3. Display the final URLs
    print(f" * 🚀 Flask App running on: http://127.0.0.1:5000")
    print(f" * 🌐 Publicly accessible at: {public_url}")
    print("\n--- Frontend UI will display below. Click the public URL to open in a new tab ---")

    # 4. Run the Flask app.
    # Because nest_asyncio.apply() was run in cell 1, this runs cleanly in Colab.
    app.run(port=5000)

except Exception as e:
    print(f"\n🚨 Connection Error: {e}")
    print("If you see an authentication error, ensure your NGROK_AUTH_TOKEN in Cell 1 is correct.")

ERROR:pyngrok.process.ngrok:t=2025-11-17T10:37:55+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: The authtoken you specified does not look like a proper ngrok authtoken.\nYour authtoken: YOUR_REAL_NGROK_AUTH_TOKEN_HERE\nInstructions to install your authtoken are on your ngrok dashboard:\nhttps://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_105\r\n"



🚨 Connection Error: The ngrok process errored on start: authentication failed: The authtoken you specified does not look like a proper ngrok authtoken.\nYour authtoken: YOUR_REAL_NGROK_AUTH_TOKEN_HERE\nInstructions to install your authtoken are on your ngrok dashboard:\nhttps://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_105\r\n.
If you see an authentication error, ensure your NGROK_AUTH_TOKEN in Cell 1 is correct.
